In [1]:
from utils import seed_torch
from train import main
import train_configs
import models

/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1802: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1828: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [2]:
config1 = train_configs.config16.config
# config2 = train_configs.config10.config

seed_torch(config1.seed)

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/kaor/.conda/envs/openmmlab/bin/python
3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) 
[GCC 9.4.0]
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)


In [4]:
import torch
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from models.eff import get_model
from runner import *
from post_processing import *
from datasets import *
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
     data = bz2.BZ2File(file, 'rb')
     data = cPickle.load(data)
     return data

In [5]:
train = pd.read_csv(config1.kfold_csv)
test = pd.read_csv('../../whale/sample_submission.csv')

In [6]:
def emb_fn_test(valid_loader, model, device,with_file_name = False):
    model.eval()
    emb = []
    targets = []
    start = end = time.time()
    for step, batch in enumerate(tqdm(valid_loader)):
        # measure data loading time
        images = batch['image'].to(device)
        if not with_file_name:
            labels = batch['label'].to(device).long()
            labels2 = batch['species'].to(device).long()
        batch_size = images.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model.extract_feature(images).cpu().numpy()
            emb.append(y_preds)
            if not with_file_name:
                targets.append(labels.cpu().numpy())
            else:
                targets.append(batch['file_name'])
    emb = np.concatenate(emb)
    targets = np.concatenate(targets)
    
    return emb,targets

In [8]:
name_dict = {}
for row in train[['individual_id','classes']].drop_duplicates().values:
    name_dict[row[1]]=row[0]

for config in [config1]:
    valid_dataset = TrainDataset(train, config.data_root_path,transform=config.val_transforms,
                                 crop_p=1.0,crop_csv_path=config.crop_csv_path,crop_backfin_csv_path=config.crop_backfin_csv_path,mode='test')
    valid_loader = DataLoader(valid_dataset, 
                                  batch_size=32, 
                                  shuffle=False, 
                                  num_workers=4, 
                                  pin_memory=True, 
                                  drop_last=False)
    test_dataset = TestDataset(test, config.data_root_path.replace("train_images","test_images"),transform=config.val_transforms,
                               crop_p=1.0,crop_csv_path='/home/kaor/whale_code/yolo/test_bbox.csv',
                               crop_backfin_csv_path='/home/kaor/whale_code/yolo/test_bbox_backfin.csv')
    test_loader = DataLoader(test_dataset, 
                                  batch_size=32, 
                                  shuffle=False, 
                                  num_workers=4, 
                                  pin_memory=True,drop_last=False)
    for fold in range(5):
        model_path = config.save_dir+f'{config.exp_name}_fold{fold}_best.pth'
        if not os.path.exists(model_path):
            continue
        if os.path.exists(f"{config.exp_name}_fold{fold}.csv"):
            continue
        print(model_path)
        model = eval(config.model['f_class'])(**config.model['args']).to(config.device)
        model.load_state_dict(torch.load(model_path)['model'])
        emb_test,targets_filename = emb_fn_test(test_loader, model, config.device,with_file_name = True)
        emb_val,targets_val = emb_fn_test(valid_loader, model, config.device,with_file_name = False)
        data = {'emb_val':emb_val,"targets_val":targets_val,"emb_test":emb_test,"targets_filename":targets_filename}
        compressed_pickle(config.save_dir+f'{config.exp_name}_fold{fold}', data)
        EMB_SIZE = 512
        vals_blend = []
        labels_blend = []
        inds_blend = []
        for i in range(1):    
            vals, inds = get_topk_cossim_sub(emb_test[:,i*EMB_SIZE:(i+1)*EMB_SIZE], emb_val[:,i*EMB_SIZE:(i+1)*EMB_SIZE], k=1500)
            vals = vals.data.cpu().numpy()
            inds = inds.data.cpu().numpy()
            labels = np.concatenate([targets_val[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)
            vals_blend.append(vals)
            labels_blend.append(labels)
            inds_blend.append(inds)
        df = []
        for i,t in enumerate(targets_filename):
            res = []
            cc= labels_blend[0][i]
            for c in cc:
                tl = name_dict[c]
                if tl not in res:
                    res.append(tl)
                    if 'new_individual' not in res:
                        res.append('new_individual')
                    if len(res)==5:
                        break
            df.append([t," ".join(res)])
        pd.DataFrame(df,columns=test.columns).to_csv(f"{config.exp_name}_fold{fold}.csv",index=False)

/home/kaor/whale_code/output/exp16_fold0_best.pth


 77%|███████▋  | 1232/1595 [14:34<04:12,  1.44it/s]

/home/kaor/whale/train_images//67f849396b6aaf.jpg


100%|██████████| 437/437 [00:00<00:00, 447.84it/s]


In [27]:
# th = vals_blend[0][:,0].mean()
# vals_blend[0][:,0].min()

0.28598335

In [30]:
# df = []
# for i,t in enumerate(targets_filename):
#     res = []
#     cc= labels_blend[0][i]
#     dist = vals_blend[0][i]
#     for c,di in zip(cc,dist):
#         tl = name_dict[c]
#         if tl not in res:
#             res.append(tl)
#             if 'new_individual' not in res and di<(th-.1):
#                 res.append('new_individual')
#             if len(res)==5:
#                 break
#     df.append([t," ".join(res)])
# pd.DataFrame(df,columns=test.columns).to_csv(f"{config.exp_name}_fold{fold}_th.csv",index=False)

In [29]:
# pd.DataFrame(df,columns=test.columns)

,image,predictions
0,000110707af0ba.jpg,fbe2b15b5481 528e2b621c73 new_individual 0a870...
1,0006287ec424cb.jpg,1424c7fec826 new_individual c3b7d902e73c f0efb...
2,000809ecb2ccad.jpg,1ce3ba6a3c29 new_individual b91c12ffc2b0 c0fd9...
3,00098d1376dab2.jpg,c4274d90be60 new_individual 43946ac5a865 3d22c...
4,000b8d89c738bd.jpg,cf500d3874bc new_individual bcc45b6870e5 83533...
...,...,...
27951,fff6ff1989b5cd.jpg,734573d54bef new_individual 843297d1983b ec0aa...
27952,fff8fd932b42cb.jpg,839bd24faf23 new_individual 4e376cb4fd38 53954...
27953,fff96371332c16.jpg,2fad3a13934d new_individual fd983ada9fe2 23fa4...
27954,fffc1c4d3eabc7.jpg,d85f2d5186cb new_individual a8fe10f7b3e0 e7f9e...


In [ ]:
# plt.imshow(plt.imread("/home/kaor/whale/test_images/0485e977ceedaf.jpg")[1000:1200,1500:2000])

In [ ]:
# emb_test.shape

In [ ]:
# EMB_SIZE = 512
# vals_blend = []
# labels_blend = []
# inds_blend = []
# for i in range(1):    
#     vals, inds = get_topk_cossim_sub(emb_test[:,i*EMB_SIZE:(i+1)*EMB_SIZE], emb_val[:,i*EMB_SIZE:(i+1)*EMB_SIZE], k=1500)
#     vals = vals.data.cpu().numpy()
#     inds = inds.data.cpu().numpy()
#     labels = np.concatenate([targets_val[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)
#     vals_blend.append(vals)
#     labels_blend.append(labels)
#     inds_blend.append(inds)

In [ ]:
# name_dict = {}
# for row in train[['individual_id','classes']].drop_duplicates().values:
#     name_dict[row[1]]=row[0]
# # name_dict

In [ ]:
# cc = labels_blend[0][0]
# if cc[0]!=cc[1] or cc[1]==cc[2]:
#     print(cc[0:3])

In [ ]:
# df = []
# for i,t in enumerate(targets_filename):
#     res = []
#     cc= labels_blend[0][i]
#     for c in cc:
#         tl = name_dict[c]
#         if tl not in res:
#             res.append(tl)
#             if 'new_individual' not in res:
#                 res.append('new_individual')
#             if len(res)==5:
#                 break
#     df.append([t," ".join(res)])
# pd.DataFrame(df,columns=test.columns).to_csv(f"{config.exp_name}.csv",index=False)

In [ ]:
# from sklearn.neighbors import NearestNeighbors
# neigh = NearestNeighbors(n_neighbors=500,metric='cosine')
# neigh.fit(emb_val)

In [ ]:
# df = pd.DataFrame(df,columns=test.columns)
# # for row in df['predictions']:
# #     if "new" in row:
# #         print(row)
# df

In [ ]:
# for row in train['individual_id']:
#     if "new" in row:
#         print(row)